In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from pathlib import Path

import seaborn as sns
import matplotlib.pylab as plt
import matplotlib
import matplotlib.font_manager as fm

import scipy.sparse

from pathlib import Path
DATA = Path('.').resolve().parent / 'Data'

try:
    font_location = "HANDotum.ttf"
    font_name = fm.FontProperties(fname = font_location).get_name()
    matplotlib.rc('font', family=font_name)
except:
    print("폰트 임포트 에러")

In [2]:
rv1 = pd.read_csv("../Data/review1_pre.csv", index_col=[0])
rv1 = rv1.drop_duplicates(['Author', 'BookCode'], keep='first')
rv1 = rv1[~pd.isna(rv1["Author"])]

rv_book_pivot = rv1.pivot(index='Author', columns='BookCode', values='SumRate')
rv_book_pivot = rv_book_pivot.fillna(0)
us_list = rv_book_pivot.index
book_list = rv_book_pivot.columns

In [3]:
rv1

,Title,ConRate,DesRate,Author,Date,Content,BookCode,ContentMagnitude,SumRate
0,"당신의 인생을 정리해드립니다, 내 인생도 정리해보자",5,5,lazyass,2020-10-06,저는 물욕 많고 버릴 줄 모르는 사람이에요. 게다가 정리를 잘 할줄 모르니 어떤 상...,9.791165e+12,3313.073751,10
2,당신의 인생을 정리해드립니다 #가정살림,5,5,책읽는샘,2020-10-06,2020-145 <당신의 인생을 정리해드립니다(이지영 지음/쌤앤파커스)> #가정살림...,9.791165e+12,4746.308343,10
3,[가정살림] 당신의 인생을 정리해드립니다,5,4,지훈러브,2020-10-06,싱글일때는 나름 깨끗하게 정리를 잘했다고 생각했는데 아이가 태어나고 아이 용품이 늘...,9.791165e+12,4999.028552,9
4,당신의 인생을 정리해드립니다,5,5,qaz1189,2020-10-06,"예전에는 아이들을 모두 어린이집, 유치원에 등원시키고 사부작사부작 거리면서 하나하나...",9.791165e+12,4532.811448,10
5,"당신의 인생을 정리해드립니다, 이지영, 쌤앤파커스",5,5,박공주,2020-10-06,'당신의 인생을 정리해드립니다.' 저자는 당신의 '집'이 아닌 '인생'을 정리하겠...,9.791165e+12,6483.601901,10
...,...,...,...,...,...,...,...,...,...
26476,따뜻한 반전동화! [강아지 항공사에 찾아온 무시무시한 손님],5,5,lunar1982,2020-04-02,요즘처럼 전국민이 여행에 목말라있는 때가 있었을까 싶을까요?여행의 설렘이 늘 함께하...,9.788965e+12,1748.713176,10
26478,강아지 항공사에 찾아온 무시무시한 손님 / 귀엽고 따뜻한 그림책이에요,5,5,spqcu,2020-03-31,아이들이 크면 그림책을 안볼 줄 알았는데요특히 저희 둘째는 그림그리기를 좋아해서인지...,9.788965e+12,1161.610239,10
26479,"대 반전 그림책, 강아지 항공사에 찾아온 무시무시한 손님",5,5,리프,2020-03-31,허웅 ㅠㅠ 너무 귀여워요 ㅠㅠ 어떡해 ㅠㅠ표지부터 완전 심쿵! 세젤귀 강아지들이 오...,9.788965e+12,1179.810356,10
26481,마로네랑 쿠키,5,5,pyr1218,2020-10-06,모카랑 핫초코 책과 함께 구매했는데 책이 너무 작은 사이즈라서 받아보고 놀랐고 제 ...,9.791189e+12,505.367699,10


In [8]:
rv_grouped_by_author = rv1.groupby(["Author"])
rv_grouped_by_author.count().sort_values("Title", ascending=False)

,Title,ConRate,DesRate,Date,Content,BookCode,ContentMagnitude,SumRate
Author,,,,,,,,
-,77,77,77,77,77,77,77,77
오즐,58,58,58,58,58,58,58,58
kkandol32,56,56,56,56,56,56,56,56
밀크티,49,49,49,49,49,49,49,49
myliferandom,48,48,48,48,48,48,48,48
...,...,...,...,...,...,...,...,...
마마,1,1,1,1,1,1,1,1
kimnov25,1,1,1,1,1,1,1,1
asa1116,1,1,1,1,1,1,1,1


In [9]:
user_group = rv_grouped_by_author.groups
user_1_rv = []
for user in user_group.keys() :
    if len(user_group[user]) == 1 :
        user_1_rv.append(user)
len(user_1_rv)

225

In [10]:
rv1[~rv1["Author"].isin(user_1_rv)]

,Title,ConRate,DesRate,Author,Date,Content,BookCode,ContentMagnitude,SumRate
0,"당신의 인생을 정리해드립니다, 내 인생도 정리해보자",5,5,lazyass,2020-10-06,저는 물욕 많고 버릴 줄 모르는 사람이에요. 게다가 정리를 잘 할줄 모르니 어떤 상...,9.791165e+12,3313.073751,10
2,당신의 인생을 정리해드립니다 #가정살림,5,5,책읽는샘,2020-10-06,2020-145 <당신의 인생을 정리해드립니다(이지영 지음/쌤앤파커스)> #가정살림...,9.791165e+12,4746.308343,10
3,[가정살림] 당신의 인생을 정리해드립니다,5,4,지훈러브,2020-10-06,싱글일때는 나름 깨끗하게 정리를 잘했다고 생각했는데 아이가 태어나고 아이 용품이 늘...,9.791165e+12,4999.028552,9
4,당신의 인생을 정리해드립니다,5,5,qaz1189,2020-10-06,"예전에는 아이들을 모두 어린이집, 유치원에 등원시키고 사부작사부작 거리면서 하나하나...",9.791165e+12,4532.811448,10
5,"당신의 인생을 정리해드립니다, 이지영, 쌤앤파커스",5,5,박공주,2020-10-06,'당신의 인생을 정리해드립니다.' 저자는 당신의 '집'이 아닌 '인생'을 정리하겠...,9.791165e+12,6483.601901,10
...,...,...,...,...,...,...,...,...,...
26476,따뜻한 반전동화! [강아지 항공사에 찾아온 무시무시한 손님],5,5,lunar1982,2020-04-02,요즘처럼 전국민이 여행에 목말라있는 때가 있었을까 싶을까요?여행의 설렘이 늘 함께하...,9.788965e+12,1748.713176,10
26478,강아지 항공사에 찾아온 무시무시한 손님 / 귀엽고 따뜻한 그림책이에요,5,5,spqcu,2020-03-31,아이들이 크면 그림책을 안볼 줄 알았는데요특히 저희 둘째는 그림그리기를 좋아해서인지...,9.788965e+12,1161.610239,10
26479,"대 반전 그림책, 강아지 항공사에 찾아온 무시무시한 손님",5,5,리프,2020-03-31,허웅 ㅠㅠ 너무 귀여워요 ㅠㅠ 어떡해 ㅠㅠ표지부터 완전 심쿵! 세젤귀 강아지들이 오...,9.788965e+12,1179.810356,10
26481,마로네랑 쿠키,5,5,pyr1218,2020-10-06,모카랑 핫초코 책과 함께 구매했는데 책이 너무 작은 사이즈라서 받아보고 놀랐고 제 ...,9.791189e+12,505.367699,10
